In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and will be used for training.")
    except RuntimeError as e:
        print(f"Error setting up GPU: {e}")
else:
    print("No GPU detected. Training will use the CPU.")    

2025-01-30 11:04:26.052254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738215266.065295   10343 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738215266.069477   10343 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 11:04:26.083582: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU is available and will be used for training.


In [2]:
import numpy as np
import json
import os
import tensorflow as tf
import onnx

In [3]:
def save_tensor_as_bin(tensor, filepath):
    np_array = tensor.numpy() if isinstance(tensor, tf.Tensor) else tensor
    np_array.tofile(filepath)

In [4]:
# Save JSON file
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

In [5]:
# Create directories
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [6]:
def layer_wise_dump_tf(model, input_tensor, save_dir):
    # Ensure save directory structure
    create_dir(save_dir)
    create_dir(f"{save_dir}/input")
    create_dir(f"{save_dir}/output")
    create_dir(f"{save_dir}/weights")
    create_dir(f"{save_dir}/reference")

    intermediate_outputs = {}  # To store outputs for sequential execution

    # Run input through the model layer by layer
    x = input_tensor
    for layer in model.layers:
        layer_name = layer.name

        # Save input tensor to file
        save_tensor_as_bin(x, f"{save_dir}/input/{layer_name}_input.bin")

        # Save weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                save_tensor_as_bin(weight.numpy(), f"{save_dir}/weights/{layer_name}_{weight_name}.bin")

        # Compute layer output
        x = layer(x)

        # Save output tensor to file
        save_tensor_as_bin(x, f"{save_dir}/output/{layer_name}_output.bin")

        # Save layer attributes
        attributes = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,
            "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
            "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
        }
        if hasattr(layer, 'kernel_size'):
            attributes["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            attributes["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            attributes["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            attributes["activation"] = layer.activation.__name__

        save_json(attributes, f"{save_dir}/reference/{layer_name}_attributes.json")

        # Store intermediate outputs for sequential execution
        intermediate_outputs[layer_name] = x.numpy()

    return intermediate_outputs

In [7]:
model = tf.keras.models.load_model(r"file.h5")

ValueError: No model config found in the file at file.h5.

In [8]:
input_tensor = tf.random.normal([1, 32, 32, 3])  # Random input for CIFAR-10
layer_wise_dump_tf(model, input_tensor, save_dir="data")

{'conv2d': array([[[[ 2.1216761e-01, -1.2732673e-01,  4.6306229e-01, ...,
            5.3033162e-02, -1.2843771e-01, -3.7922871e-01],
          [ 6.7470157e-01, -4.8250172e-01,  1.6905934e-01, ...,
            2.5457528e-01, -7.4043852e-01, -7.4708305e-02],
          [-1.7068598e-01, -3.2297927e-01,  1.4609300e-01, ...,
           -1.8402542e-01, -7.9381108e-01,  2.0139766e-01],
          ...,
          [-8.3011150e-01,  8.1542754e-01,  4.9102597e-02, ...,
           -5.1781696e-01,  1.6300528e-01,  1.7519988e-01],
          [-1.8702525e-01,  2.7538779e-01, -1.6157538e-01, ...,
            4.1397959e-01,  1.3060243e-01,  1.2693357e-01],
          [-1.3051891e-01, -4.1753176e-01,  2.8973362e-01, ...,
            2.1223807e-01,  1.9371945e-01, -1.0191887e-01]],
 
         [[-3.4013799e-01, -6.3065016e-01, -7.6319480e-01, ...,
           -1.1111132e+00,  9.4817823e-01, -1.8749724e-01],
          [ 1.4352195e+00, -4.3347813e-03, -1.0356299e+00, ...,
           -1.2331235e+00,  1.1770820e-0

In [9]:
def generate_layer_config(model, save_dir, config_file_path):
    layer_configs = []  # List to store layer-wise configurations

    for layer in model.layers:
        layer_name = layer.name

        # Construct layer configuration
        layer_config = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,
            "input_file_path": f"{save_dir}/input/{layer_name}_input.bin",
            "output_file_path": f"{save_dir}/output/{layer_name}_output.bin",
            "weights_file_paths": [],  # To be filled if the layer has weights
            "attributes": {
                "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
                "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
            }
        }

        # Add weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                weight_file_path = f"{save_dir}/weights/{layer_name}_{weight_name}.bin"
                layer_config["weights_file_paths"].append(weight_file_path)

        # Add other layer-specific attributes if available
        if hasattr(layer, 'kernel_size'):
            layer_config["attributes"]["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            layer_config["attributes"]["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            layer_config["attributes"]["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            layer_config["attributes"]["activation"] = layer.activation.__name__

        # Append this layer's config to the list
        layer_configs.append(layer_config)

    # Save all configurations to a JSON file
    with open(config_file_path, 'w') as config_file:
        json.dump({"layers": layer_configs}, config_file, indent=4)

    print(f"Configuration file saved to {config_file_path}")


# Example usage
generate_layer_config(model, save_dir="data", config_file_path="data/config.json")


Configuration file saved to data/config.json
